이번 장에서는 고성능 함수형 프로그램을 작성할 때 사용할 수 있는 최적화 기법을 몇 가지 알아본다. 메모이제이션 알고리즘을 구현하는 방법에는 여러 가지가 있다. 또한 데코레이터를 작성하는 방법도 설명할 것이다. 더 중요한 것은 Callable 객체를 사용해 메모이제이션 한 결과를 캐시에 저장하는 방법을 보게될 것이라는 점이다.

### 메모이제이션과 캐싱

설계를 바꾸려면 약간의 생각이 핋요하다. 이에 대해 메모이제이션을 사용한 버전은 상당히 빠르지만, 많은 생각을 하여 설계를 바꿀 필요가 없다. 

### 메모이제이션 특화시키기

메모이제이션 핵심 아이디어는 @lru_cache 데코레이터에 넣을 수 있을 만큼 간단하다. 

여러 내부 캐시를 포함하는 Callable 객체를 만들 것이다. 

In [1]:
from functools import reduce
from operator import mul
prod = lambda x: reduce (mul, x)

다음은 prod 함수를 사용하는 캐시가 둘 있는 Callable 객체다.

In [7]:
from collections.abc import Callable
class Binomial(Callable):
    def __init__(self):
        self.fact_cache = {}
        self.bin_cache = {}
    def fact(self, n):
        if n not in self.fact_cache:
            self.fact_cache[n] = prod(range(1, n+1))
        return self.fact_cache[n]
    def __call__(self, n, m):
        if (n, m) not in self.bin_cache:
            self.bin_cache[n, m] = self.fact(n) // (self.fact(m)*self.fact(n-m))
        return self.bin_cache[n, m]

In [8]:
binom = Binomial()
binom(52, 5)

2598960

### 꼬리재귀 최적화

재귀를 설계하려면 다음 명령을 실행해야 한다.

In [10]:
def fact(n):
    if n == 0: return 1
    else: return n*fact(n-1)

In [11]:
def fact(n):
    if n ==0: return 1
    f = 1
    for i in range(2, n):
        f = f*1
    return f

그 어떤 최적화보다 먼저 확인해야 할 것은 해당 함수가 잘 작동하는지의 여부다. 이를 확인하기 위해 간단한 독테스트 문자열만으로 충분한 경우도 많다. 계승 함수에 다음과 같은 설명을 덧붙일 수 있을 것이다.

In [12]:
def fact(n):
    """Recursive Factorial
    >>> fact(0)
    1
    >>> fact(1)
    1
    >>> fact(7)
    5040
    """
    if n == 0: return 1
    else: return n*fact(n-1)

좀 더 복잡한 함수 조합을 사용한다면, 다음과 같은 명령을 실행해야 할 수도 있다.

In [14]:
test_example= """
>>> binom = Binomial()
>>> binom(52, 5)
2598960
"""

__test__ = {
    "test_example":test_example,
}

\__test__ 변수는 doctest.testmod() 함수가 사용하는 것이다. \__test__ 변수와 연관된 딕셔너리에 있는 모든 값도 독테스트 문자열처럼 검색 대상이 된다. 이를 통해 여러 함수를 합성한 특징을 테스트할 수 있다. 이를 통합테스트라고 부른다. 

일련의 테스트가 포함된 코드가 있다면 최적화 시 자신감을 가질 수 있고, 최적화의 올바름을 쉽게 확인할 수 있다. 다음은 최적화를 표현할 때 자주 쓰이는 유명한 인용문이다.

* 잘못된 프로그램을 더 나쁘게 만드는 것의 죄악이다.

여기서 중요한 것은 올바른 코드만을 최적화해야 한다는 것이다. 

### 메모리 최적화 

파이썬에서 사용할 수 있는 메모리 최적화 기법 중 하나는 반복 가능 객체를 사용하는 것이다. 반복 가능 객체는 실체화한 컬렉션의 특성을 상당수 포함하지만, 컬렉션이 사용하는 만큼 메모리를 쓸 필요는 없다. 반복 가능 객체에 대해 사용할 수 없는 연산이 극소수 있다. 예로 len()이 있다. 

### 정확도 최적화

계산의 정확도를 최적화할 필요가 있는 경우도 약간 존재한다. 정확도 최적화는 상당히 어려우며, 선택학 접근 방법의 정확도의 한계를 결정하려면 상당한 고등 수학이 필요할 수도 있다.

파이썬에서 할 수 잇는 재미있는 정확도 최적화로는 부동 소수점 수를 사용한 근사값 fractions.Faction 값으로 바꾸는 것이다. 일부 애플리케이션의 경우, 이렇게 하면 부동 소수점 수의 가수보다 더 많은 비트를 분모와 분자에 사용하기 때문에 훨씬 더 정확한 결과를 만들 수 있다.

decimal.Decimal 값을 정확한 계산이 필요한 경우에 사용할 수 있다는 점이 중요하다. 그러한 경우 float 값을 사용하는 것은 흔히 저지르는 실수다. float 값을 사용하면 입력의 10진 소수 값과 그 값을 2진수로 근사한 부동 소수점 수 사이의 차이로 인해 잡은 비트가 들어갈 수 있다. Decimal 값을 사용하면 이러한 매우 작업 차이가 발생하는 것을 막을 수 있다.

초월 함수를 사용하는 경우라면 이렇게 변경해도 그렇게 이익이 없을 것이다. 초월 함수는 정의에 의해 무리수로 이뤄져 있다.

### 고객의 요구에 맞춰 정확도를 감소시키기

일부 계산에서는 부동 소수점 수보다 분수를 더 직관적으로 이해할 수 있는 경우가 있다. 이는 통계 계산 결과를 고객이 이해하고 행동을 취할 수 있는 방식으로 제시하는 거의 일부분이다.

예를 들어, 카이 제곱 검정에는 보통 셀지와 예상 값 사이의 차이르 계산하는 과정이 들어간다. 그 후 그 비교값을 χ2 누적 분포 함수를 가지고 검증한다. 예상 값과 실제 값 사이에 아무 관계가 성립하지 않는다면 이를 일컬어 무관하다고 할 수 잇다. 분산 값이 임의적일 것이다. 따라서  χ2 값은 작아지는 경향이 있다. 귀무가설을 받아들인다면 어디선가는 관계가 잇어야 한다. 실제 값과 예상 값 사이의 차이가 크다면, 이 귀무가설을 기각할 수 있다. 귀무가설을 기각함으로써, 둘 사이의 관계의 더 정확한 특성을 결정하기 위해 한 걸음 더 나갈 수 있다.

이러한 결정은 종종 선택된  χ2 값과 자유두오 따른  χ2 누적 분포 함수(CDF)표를 바탕으로 이뤄진다. 표에 있는 CDF 값은 대부분 무리수이지만, 보통은 소수점 이하 2자리나 3자리 정도만 사용한다. 이 표는 의사 결정을 지원하기 위한 도구이기 때문에 0.049와 0.05 사이에는 실용적인 차이가 없다.

귀무가설을 기각하기 위해 널리 사용하는 확률은 0.05이다. 이는 1/20보다 작은 Fraction 객체다. 데이터를 고객에게 제시할 겨웅, 때로 결과를 분수로 표현하는 것이 효과적이다. 0.05와 같은 값은 시각화하기가 어렵다. 어떤 관계가 20분의 1정도 서로 관계가 있다고 이야기한다면, 상관관계의 유사성이 얼마나 되는지 이해하는 데 도움이 될 수 있다.

### 사례 분석 - 카이 제곱을 사용한 결정

일반적인 통계 결정 방법을 살펴본다.

이는 카이 제곱을 사용한 결정 방법으로 어던 데이터의 분포가 임의적인지를 판단한다. 이러한 결정을 내리기 위해서는 예상 분포를 계산하여 관찰한 데이터와 예상 값을 비교할 필요가 있다. 둘 사이에 차이가 크다면 좀 더 연구할 필요가 있다는 의미다. 둘 사이의 차이가 크지 않다면 더 이상의 연구가 필요 없다는 귀무가설을 수용할 수 있다. 이 둘의 차이는 임의적인 분산이 존재하는지의 여부뿐이다.

데이터를 파이썬으로 처리하는 방법을 본다. 몇 가지 이야기를 먼저 시작할 것이다. 이는 사례 분석의 일부는 아니지만, 탐색적 자료 분석 애플리케이션의 특성이 되곤 한다. 우너데이터를 수집하고 분석할 수 잇는 요약을 생성해야 한다. 

현업 수준의 확인 작업을 진행하는 도중에 실리콘 웨이퍼의 결함 데이터를 데이터 베이스에 수집했다. SQL 질의를 사용해 추가 분석에 필요한 겸할 상세 정보를 뽑을 수 있을 것이다. 예를 들어 당므과 같은 질의를 던질 수 있다.

SELECET SHIFT, DEFECT_CODE, SERIAL_NUMBER FROM some_tables;

이 질의의 출력은 개별 결함의 상제 정보가 들어 있는 CSV 파일이 될 수도 있다.

SQL 질의 수준에서 COUNT 나 GROUP BY문을 사용해 이를 요약할 수도 있다. 또한 파이썬 애플리케이션 수준에서 요약할 수도 있다. 완전히 데이터베이스만 사용해 요약하는 것이 더 효율적이라는 설명이 많이 있지만, 항상 그러한 것은 아니다. 원 데이터를 단순한 형태로 가져와서 파이썬 애플리케이션이 이를 요약하는 것이 SQL에서 요약을 얻는 것보다 더 빠른 경우도 있다. 성능이 중요하다면, 데이터베이스가 더 빠를 것이라고 믿고 진행하기보다는 두 가지 방법 모두 측정해봐야 한다.

데이터베이스에서 요약 정보를 효율적으로 얻을 수 있는 경우도 있다. 이러한 요약 정보에 교대 근무 시간, 결함 유형, 관찰된 결함 개수가 들어가야만 한다.

### 원데이터를 Counter 객체를 사용해 걸러내고 축약하기

collections.Counter 매개변수로 필수적인 결함 회수를 표현할 것이다. 자세한 원본 데이터로부터 교대 시간과 결함 종류별 회수를 만들 것이다. 다음은 CSV 파일에서 원데이터를 읽는 함수다.

In [17]:
import csv
from collections import Counter
from types import SimpleNamespace

def defect_reduce(input):
    rdr = csv.DictReader(input)
    assert sorted(rdr.fieldnames) == ['defect_type', 'serial_number', 'shift']
    row_ns = (SimpleNamespace(**row) for row in rdr)
    defects = ((fow.shift, row.defect_type) for row in rows_ns if row.defect_type)
    tally = Counter(defects)
    return tally

이 함수는 input 매개변수로 주어진 열려 있는 파일을 읽는 딕셔너리 리더를 만든다. 각 열의 이름이 우리가 원하는 것과 일치하는지 확인한다. 경우에 따라 열이 더 많이 있을 수도 있다. 각 열의 이름이 들어간 튜플을 가지고, 필요한 모든 열이 원본 파일에 있는지 확인한다. 

각 행에 대해 types.SimpleNamespace 매개변수를 만들었다. 앞의 예에서는 각 열의 이름이 파이썬 변수 이름에 사용할 수 있는 문자열이었기 때문에 수비게 딕셔너리를 이름 공간으로 바꿀 수 있었다. 경우에 따라서는 열의 이름을 파이썬 변수 이름으로 매핑해야 할 수도 있다.

### 요약한 데이터 읽기

모든 원 데이터를 읽는 대신, 요약한 회수만 처리하는 것을 살펴볼 수 있다. 

다음은 요약 정보를 읽는 함수다.

In [18]:
def defect_counts(source):
    rdr = csv.DictReader(source)
    assert rdr.fieldnames == ['shift', 'defect_code', 'count']
    convert = map(lambda d: ((d['shift'], d['defect_code']), int(d['count'])), rdr)
    return Counter(dict(convert))

### Counter 객체에서 확률 계산하기

교대 시간과 유형에 따른 결함 확률을 계산할 필요가 있다. 예상 확률을 계산하기 위해 몇 가지 단순한 함계 계산부터 시작할 필요가 있다. 철 번째는 모든 결함의 합으로, 다음 명령을 사용해 계산할 수 있다.


In [20]:
total = sum(defects.values())

defects에 대입된 Counter 객체에 있는 값을 직접 사용해 이를 계산했다. 그 결과 표본에서 총 309건의 결함이 있다는 것을 알 수 있다.

### 다른 요약 방법

다음은 교대 시간과 결함 유병별로 결함 확률을 계산하는 방법을 보여준다.

In [22]:
from fractions import Fraction
P_shift = dict((shift, Fraction(shift_totals[shift], total)) for shift in sorted(shift_totals))
P_type = dict((type, Fraction(type_totals[type], total)) for type in sorted(type_totals))

### 예상 값을 계산하고 분할표 표시하기

예상 결함 회수는 조합한 확률이다. 교대 시간에 따른 확률과 결함 유형에 따른 확률을 곱해 이를 계산한다. 이렇게 하면 교대 ㅅ히간과 결함 유형을 조합한 모든 12가지 확률을 계산할 수 있다. 이를 관찰 결과와 비교하여, 자세한 결함 예상 회수를 계산할 수 있다.

다음은 예상 값을 계산하는 방법을 보여준다.

In [28]:
expected = dict(
    ((s, t ), P_shift[s]*p_type[t]*total) for t in P_type for s in P_shift
)

### 카이 제곱 값 계산하기

In [31]:
diff = lambda e, o: (e-o)**2/e
chi2= sum(diff(expected[s, t], defect[s, t]) for s in shift_totals for t in type_totals)

계산을 최적화하기 위해 작은 lambda를 정의했다. 예상 값이 두 번 쓰이지만, 이 람다를 사용하면 expected[s,t]와 defects[s,t] 애트리뷰트를 단 한 번만 실행할 수 있다.

### 카이 제곱 문턱 값 계산하기

χ2 테스트의 핵심은 귀무가설을 기각하거나 채택하기 위해 받아들이고자 하는 불확실성의 정도와 자유도에 따른 문턱 값에 있다. 전통적으로 1/20라느 문턱 값을 사용해 귀무가설을 기각하라는 의견이 많다. 이는 데이터 20개 중 1개만 임의적이며, 나머지 데이터는 의미가 있다는 뜻이다. 다른 말로 하면, 20번 중 19번은 데이터가 단순한 임의적인 변동을 표현할 것으로 생각한다는 뜻이다.

카이 제곱 값은 초월 함수를 몇 가지 계산해야 하기 때문에 보통 표 형태로 제공된다. 라이브러리에서 χ2 누적 분포 함수를 제공한다면, 중요한 값을 찾기 위해 표를 찾는 대신 값을 계산할 수 있다.

자유도 k인 χ2값 x에 대한 누적 분포 함수는 다음과 같이 정의된다.

$$F(x; k) = \frac{\gamma(\frac{k}{2}, \frac{x}{2})} {\Gamma(\frac{k}{2})}$$

임의적인 확률을 $p = 1 - F(\chi^2;k)$로 두는 경우가 일반적이다. 따라서 p > 0.005면 데이터를 임의적인 것으로 이해할 수 이쏙, 귀무가설이 참이 된다는 뜻이다.

이를 위해서는 불완전한 감마 함수인 $\gamma(s, x)$와 완전한 감마 함수인  $\Gamma(x)$를 계산해야 한다. 이들은 상당히 복잡한 수학을 필요로 한다. 일부 특별한 경우를 생략하면, 이렇나 계산에 딱 들어맞는 상당히 좋은 근사 함수를 구현할 수 있다. 이 두 함수를 함수형 설계의 여러 가지 문제를 보여준다. 

두 함수 모두 계승 계산(n!)을 필요로 한다. 이 주제에 대해 여러 가지 군현을 이미 살펴봤다. 여기서는 다음 정의르르 사용할 것이다.

In [34]:
from functools import lru_cache
import operator
@lru_cache(128)
def fact(k):
    if k < 2: return 1
    return reduce(operator.mul, range(2, int(k)+1))

이는 $k! =\prod_{2\leq i\leq k}$ 2부터 k까지의 값을 모두 곱한 것이라는 의미다. 

### 부분 감마 함수 계산하기

부분 감마 함수는 단순한 수열의 확장이다. 

이 수열은 언젠가는 너무 작아서 중요하지 않게 되는 시점까지 작아진다. 

수열의 각 항은 s=1이고, z=2일 때 다음과 같다.

2/1, -2/1, 4/3, -2/3, 4/15, -4/45, ... , -2/638512875

In [35]:
def gamma(s, z):
    def terms(s, z):
        for k in range(100):
            t2 = Fraction(z**(s+k)) / (s+k)
            term = Fraction((-1)**k, fact(k))*t2
            yield term
        warning.warn("More than 100 terms")
        
    def take_until(function, iterable):
        for v in iterable:
            if function(v): return
            yield v
    e = 1E-8
    return sum(take_until(lambda t: abs(t) < e, terms(s,z)))

term: 수열의 각 함을 만들어 낸다. 루프에 상환은 100으로 지정했다. itertools.count를 사용해 항의 무한 시퀀스를 만들 수도 있다. 
$z^(s+k)$: 무리수 값으로 Fraction 값을 만들었다. 
take_until: 반복 가능 객체로부터 주어진 함수가 참이 될 때까지 취한다.

### 완전한 감마 계산하기

완전환 감마 함수는 좀 더 어렵다. 파이썬 수학 라이브러리도 완전한 감마 함수를 제공한다. 그 함수는 여러 상황을 위해 설계한 여러 분야에 유용한 근사 함수다. 

완전한 감마 함수의 완전한 구현에는 관심이 없다. 두 가지 특별한 경우에 대해서만 관심이 있을 뿐이다. 한 가지는 정수 값이며, 다른 한 가지는 1/2로 끝나는 수다. 이 두가지 특별한 경우에 대해서는 정확한 답을 구할 수 있으며, 근사값을 구할 필요가 없다.

정수 값의 경우 $\Gamma{n=n(n-1})!$이다. 주어진 정수에 대한 감마 함수는 앞에서 정의한 계승 값을 사용해 계산할 수 있다.

1/2로 끝나는 수에 대해서는 특별한 형태가 있다.

$$\Gamma{\frac{1}{2} + n} = \frac{(2n)!}{4^n n!} \sqrt{\pi}$$

이 값에는 무리수가 들어 있다. 따라서 이를 float나 Fraction 객체를 사용해 근사값으로 표현할 수 있다.

카이 제곱 누적 분포 함수는 다음에 설명한 완전한 감마 함수의 두 가지 특성을 활용하기 때문에 일반적인 감마 함수는 불필요하다. 따라서 약간의 속임수를 써서, 적당히 정확한 후 두 갑을 사용할 수 있다.

정당한 Fraction 값을 사용하면, 분모가 1인 경우(즉, 정수)와 분모가 2인 경우를 처리하는 함수를 설계할 수 있다. 다음과 같이 Fraction값을 활용할 수 있다.

In [38]:
sqrt_pi = Fraction(677622787, 382307718)
def Gamma_Half(k):
    if isinstance(k, int):
        return fact(k-1)
    elif isinstance(k, Fraction):
        if k.denominator == 1:
            return fact(k-1)
        elif k.denominator == 2:
            n = k-Fraction(1,2)
            return fact(2*n) / (Fraction(4**n)*fact(n))*sqrt_pi
    raise ValueError("Can't compute 감마({0})". format(k))

오직 정수와 끝이 1/2인 경우만으로 다룬다는 의미를 가오잫기 위해 이름을 Gamme_Half라고 붙였다. 정수의 경우 fact 함수를 사용한다.

In [42]:
g = Gamma_Half(Fraction(3,2))
g.limit_denominator(2000000)

Fraction(291270, 328663)

이렇게 하면 분모 값을 1부터 2백만 사이로 제한할 수 있다. 따라서 단위 테스트에 사용하기 쉬운, 보기 편한 여섯 자기 숫자를 얻을 수 있다. 

### 임의적인 분포의 확률 계산하기

이제 불완전한 감마 함수이 gamma와 완전한 감마 함수인 Gamma_Half가 있으므로 $\chi{^2} CDF$ 값을 계산할 수 있다. 주어진 카이제곱 값이 임의적이거나 어떤 상관관게가 존재할 확률을 보여준다. 

In [47]:
def cdf(x, k):
    return 1- gamma(Fraction(k,2), Fraction(x/2))/Gamma_Half(Fraction(k,2))

In [48]:
round(float(cdf(0.004, 1)), 2)

0.95

In [49]:
cdf(0.004, 1).limit_denominator(100)

Fraction(94, 99)

In [50]:
round(float(cdf(10.83,1)), 3)

0.001

In [51]:
round(float(cdf(3.94, 10)), 2)

0.95